## Libraries

In [1]:
# Standard libraries
import itertools as it

# Optional libraries for dev purposes
import os
import sys

import nibabel as nib
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import math
# Python implementation of Matlab's imresize due to differing results
# Original code found here: https://github.com/fatheral/matlab_imresize
# Note this code was last updated Aug. 2021
USER="ataha" # Update this to or path below to where imresize.py is located
sys.path.append(f'/home/ROBARTS/ataha/Desktop/auto-afids/autofid_main/training/')
from imresize import *

## Functions

In [2]:
def read_nii_metadata(nii_path):
    """Load in nifti data and header information and normalize MRI volume"""
    nii = nib.load(nii_path)
    nii_affine = nii.affine
    nii_data = nii.get_fdata()
    #normalization 
    nii_data = (nii_data - nii_data.min())/ (nii_data.max() - nii_data.min())
    return nii_affine, nii_data

In [3]:
def get_fid(fcsv_path, fid_num):
    """Extract specific fiducial's spatial coordinates"""
    fcsv_df = pd.read_csv(fcsv_path, sep=",", header=2)

    return fcsv_df.loc[fid_num, ["x", "y", "z"]].to_numpy(dtype="single")

In [4]:
def fid_world2voxel(fid_world, nii_affine, resample_size=1, padding=None):
    """Transform fiducials in world coordinates to voxel coordinates

    Optionally, resample to match resampled image
    """

    # Translation
    fid_voxel = fid_world.T - nii_affine[:3, 3:4]
    # Rotation
    fid_voxel = np.dot(fid_voxel, np.linalg.inv(nii_affine[:3, :3]))

    # Round to nearest voxel
    fid_voxel = np.rint(np.diag(fid_voxel) * resample_size)

    if padding:
        fid_voxel = np.pad(fid_voxel, padding, mode="constant")

    return fid_voxel.astype(int)

In [5]:
def image_resize(img, size):
    """Resize image using Matlab's imresize"""
    return imresize(img, size)

In [6]:
def integral_volume(resampled_image):
    '''Compute zero-padded resampled volume'''
    iv_image = resampled_image.cumsum(0).cumsum(1).cumsum(2)
    iv_zeropad = np.zeros((iv_image.shape[0]+1, iv_image.shape[1]+1, iv_image.shape[2]+1))
    iv_zeropad[1:, 1:, 1:] = iv_image
    
    return iv_zeropad

## Variables

In [7]:
PAD_FLAG = False
PADDING = 0
SIZE = 1  # Fixed for now but make variable
sampling_rate = 5
DATASET = 'OASIS'

# Directories
BASE_DIR = f"/home/ROBARTS/{USER}/graham/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/autoafids/data/{DATASET}"
T1W_DIR = f"{BASE_DIR}/derivatives/afids_mni"
AFIDS_DIR = f"{BASE_DIR}/derivatives/afids_groundtruth"

#describes haar like feature coordiantes 
feature_offsets = f"/home/ROBARTS/{USER}/Desktop/auto-afids/autofid_main/training/feature_offsets.npz"

SUBJECT_IDS = [subject for subject in os.listdir(T1W_DIR) if "sub-" in subject]

## Code

In [8]:
for z in range(0,32): 
    FID_NUM = z +1  
    print(FID_NUM)
    finalpred = []
    for i,n in enumerate(SUBJECT_IDS):
        print(f' {n}')
        SUBJECT = n
        # Load image -- assumes correct bids spec 
        nii_path = (f"{T1W_DIR}/{SUBJECT}/{SUBJECT}_space-MNI152NLin2009cAsym_T1w.nii.gz")
        aff, img = read_nii_metadata(nii_path)

        # Get and compute new fiducial location
        fcsv_path = f"{T1W_DIR}/{SUBJECT}/{SUBJECT}_space-MNI152NLin2009cAsym_desc-groundtruth_afids.fcsv"
        fid_world = get_fid(fcsv_path, FID_NUM - 1)
        resampled_fid = fid_world2voxel(fid_world, aff, resample_size=SIZE, padding=PADDING)

        # Get image samples (sample more closer to target)
        inner_its = [
            range(resampled_fid[0] - sampling_rate, resampled_fid[0] + (sampling_rate+1)),
            range(resampled_fid[1] - sampling_rate, resampled_fid[1] + (sampling_rate+1)),
            range(resampled_fid[2] - sampling_rate, resampled_fid[2] + (sampling_rate+1)),
        ]
        inner_samples = [t for t in it.product(*inner_its)]

        outer_its = [
            range(resampled_fid[0] - sampling_rate*2, resampled_fid[0] + sampling_rate*2+1, 2),
            range(resampled_fid[1] - sampling_rate*2, resampled_fid[1] + sampling_rate*2+1, 2),
            range(resampled_fid[2] - sampling_rate*2, resampled_fid[2] + sampling_rate*2+1, 2),
        ]
        outer_samples = [t for t in it.product(*outer_its)]

        # Concatenate and retain unique samples and
        all_samples = np.unique(
            np.array(inner_samples + outer_samples), axis=0)

        # Compute Haar-like features features
        # Make this optional to load or create
        feature_offsets_data = np.load(feature_offsets)
        smin, smax = feature_offsets_data["arr_0"], feature_offsets_data["arr_1"]

        # Generate bounding cube surrounding features
        min_corner_list = np.zeros((4000 * all_samples.shape[0], 3)).astype('uint8')
        max_corner_list = np.zeros((4000 * all_samples.shape[0], 3)).astype('uint8')

        for idx in range(all_samples.shape[0]):
            min_corner_list[idx*4000 : (idx+1)*4000] = all_samples[idx] + smin
            max_corner_list[idx*4000 : (idx+1)*4000] = all_samples[idx] + smax

        corner_list = np.hstack((min_corner_list, max_corner_list))

        #compute the integral image for more efficient generation of haar-like features
        iv_image = integral_volume(img)

        #intialize a numpy array to store features
        testerarr = np.zeros((4000 * all_samples.shape[0])) 

        # Generate features using integral volume
        numerator = (
            iv_image[corner_list[:, 3] + 1, corner_list[:, 4] + 1, corner_list[:, 5] + 1]
            - iv_image[corner_list[:, 0], corner_list[:, 4] + 1, corner_list[:, 5] + 1]
            - iv_image[corner_list[:, 3] + 1, corner_list[:, 4] + 1, corner_list[:, 2]]
            - iv_image[corner_list[:, 3] + 1, corner_list[:, 1], corner_list[:, 5] + 1]
            + iv_image[corner_list[:, 3] + 1, corner_list[:, 1], corner_list[:, 2]]
            + iv_image[corner_list[:, 0], corner_list[:, 1], corner_list[:, 5] + 1]
            + iv_image[corner_list[:, 0], corner_list[:, 4] + 1, corner_list[:, 2]]
            - iv_image[corner_list[:, 0], corner_list[:, 1], corner_list[:, 2]]
        )

        denomerator = (
            (corner_list[:, 3]-corner_list[:, 0]+1)
            *(corner_list[:, 4]-corner_list[:, 1]+1)
            *(corner_list[:, 5]-corner_list[:, 2]+1)
        )

        #dump features into intialized variable
        testerarr = numerator/denomerator
        vector1arr = np.zeros((4000 * all_samples.shape[0]))
        vector2arr = np.zeros((4000 * all_samples.shape[0]))

        for index in range(all_samples.shape[0]):
            vector = range(index * 4000, index * 4000 + 2000)
            vector1arr[index * 4000 : (index + 1) * 4000 - 2000] = vector

        for index in range(all_samples.shape[0]):
            vector = range(index * 4000 + 2000, index * 4000 + 4000)
            vector2arr[index * 4000 + 2000 : (index + 1) * 4000] = vector

        vector1arr[0] = 1
        vector1arr = vector1arr[vector1arr != 0]
        vector1arr[0] = 0
        vector2arr = vector2arr[vector2arr != 0]
        vector1arr = vector1arr.astype(int)
        vector2arr = vector2arr.astype(int)

        diff = testerarr[vector1arr] - testerarr[vector2arr]
        diff = np.reshape(diff, (all_samples.shape[0], 2000))
        dist = all_samples - resampled_fid[0:3]
        p = np.sqrt(dist[:, 0] ** 2 + dist[:, 1] ** 2 + dist[:, 2] ** 2)
        p = np.exp(-0.5*p) #weighted more towards closer distances 

        for index in range(p.shape[0]):
            finalpred.append(np.hstack((diff[index], p[index])))
    
    finalpred = np.asarray(finalpred)
    print(finalpred.shape)
    # Concatenate to array of feature vectors.
    finalpredarr = np.zeros((1, 2001))
    finalpredarr = np.concatenate((finalpredarr, finalpred))
    # Model training.
    finalpredarr = finalpredarr[1:, :]
    regr_rf = RandomForestRegressor(
        n_estimators=20,
        max_features=0.33,
        min_samples_leaf=5,
        random_state=2,
        n_jobs=-1)
    
    X_train = finalpredarr[:, :-1]
    y_train = finalpredarr[:, -1]

    print("training start")
    Mdl = regr_rf.fit(X_train, y_train)
    dump(Mdl, f'{FID_NUM}_{sampling_rate}x{sampling_rate}x{sampling_rate}.joblib') 
    print("training ended")

1
 sub-0274
 sub-0145
 sub-0284
 sub-0200
 sub-0239
 sub-0263
 sub-0177
 sub-0343
 sub-0216
 sub-0010
 sub-0101
 sub-0371
 sub-0456
 sub-0345
 sub-0256
 sub-0086
 sub-0357
 sub-0114
 sub-0249
 sub-0117
 sub-0365
 sub-0395
 sub-0255
 sub-0398
 sub-0180
(63425, 2001)
training start
training ended
2
 sub-0274
 sub-0145
 sub-0284
 sub-0200
 sub-0239
 sub-0263
 sub-0177
 sub-0343
 sub-0216
 sub-0010
 sub-0101
 sub-0371
 sub-0456
 sub-0345
 sub-0256
 sub-0086
 sub-0357
 sub-0114
 sub-0249
 sub-0117
 sub-0365
 sub-0395
 sub-0255
 sub-0398
 sub-0180
(63425, 2001)
training start
training ended
3
 sub-0274
 sub-0145
 sub-0284
 sub-0200
 sub-0239
 sub-0263
 sub-0177
 sub-0343
 sub-0216
 sub-0010
 sub-0101
 sub-0371
 sub-0456
 sub-0345
 sub-0256
 sub-0086
 sub-0357
 sub-0114
 sub-0249
 sub-0117
 sub-0365
 sub-0395
 sub-0255
 sub-0398
 sub-0180
(63425, 2001)
training start
training ended
4
 sub-0274
 sub-0145
 sub-0284
 sub-0200
 sub-0239
 sub-0263
 sub-0177
 sub-0343
 sub-0216
 sub-0010
 sub-0101
